In [ ]:
!pip install flask pyngrok transformers torch PyPDF2 pandas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.8 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `llama3-token` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [ ]:
!ngrok config add-authtoken 30DWmXnyTPR9NsYZ3YNfBSkPkgK_LteWuSr3gPC4mMmRUqGy

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Start the backend server (LLaMA 3 + Flask + ngrok)
from flask import Flask, request, jsonify, send_file
from transformers import AutoTokenizer, AutoModelForCausalLM
from pyngrok import ngrok
import torch, PyPDF2, pandas as pd, io, csv
import os

app = Flask(__name__)

# Load tokenizer and model on GPU
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)
print("Model loaded!")

def parse_pdf(file_stream):
    reader = PyPDF2.PdfReader(file_stream)
    return "\n".join([page.extract_text() for page in reader.pages])

def parse_csv(file_stream):
    df = pd.read_csv(file_stream)
    return "\n".join([f"{row['Patient']}\n{row['Therapist']}" for _, row in df.iterrows()])

def build_prompt(book_text, starter_dialogue):
    return f"""
You are a professional therapist. Based on the following therapy guide and ongoing dialogue, continue the conversation empathetically and realistically.

--- Therapy Guide ---
{book_text}

--- Conversation so far ---
{starter_dialogue}

--- Continue the conversation starting as Therapist. Respond with realistic alternating lines between Patient and Therapist until a natural stopping point. ---
"""

@app.route("/generate", methods=["POST"])
def generate():
    try:
        pdf_file = request.files["pdf"]
        csv_file = request.files["csv"]

        book_text = parse_pdf(pdf_file.stream)
        starter_dialogue = parse_csv(csv_file.stream)
        prompt = build_prompt(book_text, starter_dialogue)

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            pad_token_id=tokenizer.eos_token_id
        )

        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response_text = generated[len(prompt):].strip()

        # Turn into alternating dialogue CSV
        lines = response_text.split("\n")
        data = []
        for i in range(0, len(lines), 2):
            patient = lines[i].strip()
            therapist = lines[i+1].strip() if i+1 < len(lines) else ""
            data.append([patient, therapist])

        output_path = "/tmp/generated_dialogue.csv"
        with open(output_path, "w", newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Patient", "Therapist"])
            writer.writerows(data)

        return send_file(output_path, as_attachment=True)

    except Exception as e:
        return jsonify({"response": f"Error in Colab: {str(e)}"}), 500

# Start ngrok and Flask
public_url = ngrok.connect(5000)
print(" * ngrok tunnel URL:", public_url)
app.run()


Loading model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded!
 * ngrok tunnel URL: NgrokTunnel: "https://87b992f4dff5.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Jul/2025 21:31:41] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jul/2025 21:34:04] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jul/2025 21:51:50] "POST /generate HTTP/1.1" 200 -
